In [2]:
"""
Week 2 Assignment: LLM Engineering
Author: Nikhil Raut

Notebook: ai_domain_finder.ipynb

Purpose:
Build an agentic AI Domain Finder that proposes short, brandable .com names, verifies availability via RDAP, 
then returns: 
    a list of available .coms, 
    one preferred pick, 
    and a brief audio rationale.
"""


'\nWeek 2 Assignment: LLM Engineering\nAuthor: Nikhil Raut\n\nNotebook: ai_domain_finder.ipynb\n\nPurpose:\nBuild an agentic AI Domain Finder that proposes short, brandable .com names, verifies availability via RDAP, \nthen returns: \n    a list of available .coms, \n    one preferred pick, \n    and a brief audio rationale.\n'

In [3]:
import os
import json
import requests
from typing import Dict, List, Tuple

from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

OPENAI_MODEL = "gpt-5-nano-2025-08-07"
TTS_MODEL = "gpt-4o-mini-tts"

openai = OpenAI()

In [ ]:
RDAP_URL = "https://rdap.verisign.com/com/v1/domain/{}"

def _to_com(domain: str) -> str:
    d = domain.strip().lower()
    if d.endswith(".com"):
        return d
    return f"{d}.com"

def check_com_availability(domain: str) -> Dict:
    """
    Returns: {"domain": "name.com", "available": bool, "status": int}
    Rule: HTTP 200 => already registered (NOT available); 404 => available.
    """
    fqdn = _to_com(domain)
    try:
        r = requests.get(RDAP_URL.format(fqdn), timeout=6)
        available = (r.status_code == 404)
        return {"domain": fqdn, "available": available, "status": r.status_code}
    except requests.RequestException:
        return {"domain": fqdn, "available": False, "status": 0}


In [5]:
check_tool = {
    "type": "function",
    "function": {
        "name": "check_com_availability",
        "description": "Check if a .com domain is available using RDAP. Accepts root or full domain.",
        "parameters": {
            "type": "object",
            "properties": {
                "domain": {
                    "type": "string",
                    "description": "Domain root or FQDN to check (limited to .com)."
                }
            },
            "required": ["domain"],
            "additionalProperties": False
        }
    }
}

TOOLS = [check_tool]


In [6]:
def handle_tool_calls(message) -> List[Dict]:
    """
    Translates model tool_calls into tool results for follow-up completion.
    """
    results = []
    for call in (message.tool_calls or []):
        if call.function.name == "check_com_availability":
            args = json.loads(call.function.arguments or "{}")
            payload = check_com_availability(args.get("domain", ""))
            results.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": json.dumps(payload)
            })
    return results


In [ ]:
SYSTEM_PROMPT = """You are the Agent for project "AI Domain Finder".
Goal: suggest .com domains and verify availability using the tool ONLY (no guessing).

Instructions:
- Always propose 5-12 brandable .com candidates based on:
  (1) Industry, (2) Target Customers, (3) Description.
- For each candidate, CALL the tool check_com_availability.
- Respond ONLY after checking all candidates.
- Output Markdown with three sections and these exact headings:
  1) Available .com domains:
     - itemized list (root + .com)
  2) Preferred domain:
     - a single best pick
  3) Audio explanation:
     - 1-2 concise sentences explaining the preference

Constraints:
- Use customer-familiar words where helpful.
- Keep names short, simple, pronounceable; avoid hyphens/numbers unless meaningful.
- Never include TLDs other than .com.
"""


In [ ]:
def run_agent_with_tools(history: List[Dict]) -> str:
    """
    history: list of {"role": "...", "content": "..."} messages
    returns assistant markdown string (includes sections required by SYSTEM_PROMPT)
    """
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history
    resp = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages, tools=TOOLS)

    while resp.choices[0].finish_reason == "tool_calls":
        tool_msg = resp.choices[0].message
        tool_results = handle_tool_calls(tool_msg)
        messages.append(tool_msg)
        messages.extend(tool_results)
        resp = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages, tools=TOOLS)

    return resp.choices[0].message.content

In [ ]:
def extract_audio_text(markdown_reply: str) -> str:
    """
    Pulls the 'Audio explanation:' section; falls back to first sentence.
    """
    marker = "Audio explanation:"
    lower = markdown_reply.lower()
    idx = lower.find(marker.lower())
    if idx != -1:
        segment = markdown_reply[idx + len(marker):].strip()
        parts = segment.split(".")
        return (". ".join([p.strip() for p in parts if p.strip()][:2]) + ".").strip()
    return "This domain is the clearest, most memorable fit for the audience and brand goals."

def synth_audio(text: str) -> bytes:
    audio = openai.audio.speech.create(
        model=TTS_MODEL,
        voice="alloy",
        input=text
    )
    return audio.content


In [11]:
def chat(message: str, history_ui: List[Dict]) -> Tuple[List[Dict], bytes]:
    """
    Gradio ChatInterface callback.
    - message: latest user text (free-form)
    - history_ui: [{"role": "user"/"assistant", "content": "..."}]
    Returns: updated history, audio bytes for the 'Audio explanation'.
    """
    # Convert Gradio UI history to OpenAI-format history
    history = [{"role": h["role"], "content": h["content"]} for h in history_ui]
    history.append({"role": "user", "content": message})

    reply_md = run_agent_with_tools(history)
    history.append({"role": "assistant", "content": reply_md})

    audio_text = extract_audio_text(reply_md)
    audio_bytes = synth_audio(audio_text)

    return history, audio_bytes


In [ ]:
INTRO = (
    "Please provide details as text (three lines or paragraphs):\n"
    "Industry: ...\n"
    "Target Customers: ...\n"
    "Description: ...\n\n"
    "You can refine in follow-ups (e.g., tone, shorter names, avoid words, etc.)."
)

with gr.Blocks(title="AI Domain Finder (.com only)") as ui:
    gr.Markdown("# AI Domain Finder (.com only)")
    gr.Markdown("Provide your business details. The Agent will suggest .com options, verify availability, pick a preferred domain, and speak a short rationale.")
    with gr.Row():
        chatbot = gr.Chatbot(type="messages", height=460)
    with gr.Row():
        audio_out = gr.Audio(label="Audio explanation", autoplay=True)
    with gr.Row():
        msg = gr.Textbox(label="Your input", placeholder=INTRO, lines=6)

    def _append_user(m, hist):
        return "", hist + [{"role": "user", "content": m}]

    msg.submit(_append_user, inputs=[msg, chatbot], outputs=[msg, chatbot]).then(
        chat, inputs=[msg, chatbot], outputs=[chatbot, audio_out]
    )

ui.launch(inbrowser=True, auth=None, show_error=True)
